# Week 3 - Training

Now that we have the shape of our general purpose curve fitting function, how do we train them?

* What is training?
* Gradient descent
* Stochastic Gradient descent
* Train/test split
* Learning curves
* Hyperparameters 
    * Numebr of layers
    * Size of each layer
* Dropout
* L1/L2 regularization    
* Train our first network with Keras

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## What is Training

In the last lesson, we talked about the structure of neural networks as mathematical functions.
We learned that neural networks satisfy the universal approximation theorem, meaning that with a good choice of parameters they can approximate _any_ function.
This week, we will discuss _how_ we can find the best values for the weights and biases so that the neural network fits whatever data we like.
The process of finding the best parameters is called **training.**

## Gradient Descent

TODO

### Loss Functions for Classification and Regression

## Stochastic Gradient Descent

TODO

## Training a Neural Network with Keras


# Homework Questions

1. Describe the difference between standard gradient descent and stochastic gradient descent.
Explain the advantages of using stochastic gradient descent.

2. Explain what the purpose of a loss function is. Describe two loss functions we learned about and what the use cases would be for each.

3. Use the Pandas library to load the given dataset. Use the `train_test_split` function to split the data into training and testing datasets with 80% for training and 20% for testing.

4. Use the Keras library's `Sequential` model to build a simple neural network with one hidden layer with 100 neurons. Train the model to predict the column `___NAME___`. Evaluate the neural network on the test data you created in question 2.

5. Use learning curves to determine if the neural network from question \#4 is underfitting or overfitting. Explain your reasoning.

6. Try to improve the neural network you trained in question \#4 above by including regularization, dropout, or by varying the number or size of layers.